In [1]:
import momepy as mm
import geopandas as gpd
import libpysal

In [2]:
path = 'files/AMS/elements.gpkg'

In [3]:
tess = gpd.read_file(path, layer="tessellation")
blg = gpd.read_file(path, layer="buildings")
blocks = gpd.read_file(path, layer='blocks')

In [4]:
blg['sdbAre'] = mm.Area(blg).series
blg['sdbVol'] = mm.Volume(blg, 'sdbHei', 'sdbAre').series
blg['sdbPer'] = mm.Perimeter(blg).series
blg['sdbCoA'] = mm.CourtyardArea(blg, 'sdbAre').series

blg['ssbFoF'] = mm.FormFactor(blg, 'sdbVol', 'sdbAre').series
blg['ssbVFR'] = mm.VolumeFacadeRatio(blg, 'sdbHei', 'sdbVol', 'sdbPer').series
blg['ssbCCo'] = mm.CircularCompactness(blg, 'sdbAre').series
blg['ssbCor'] = mm.Corners(blg).series
blg['ssbSqu'] = mm.Squareness(blg).series
blg['ssbERI'] = mm.EquivalentRectangularIndex(blg, 'sdbAre', 'sdbPer').series
blg['ssbElo'] = mm.Elongation(blg).series
'''
cencon = mm.CentroidCorners(blg)
+blg['ssbCCM'] = cencon.mean
+blg['ssbCCD'] = cencon.std
'''


  0%|          | 206/252386 [00:00<21:43, 193.50it/s]/home/arch_ubuntu/Documents/mf/momepy/momepy/shape.py:797: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
  0%|          | 255/252386 [00:00<15:40, 268.03it/s]/home/arch_ubuntu/Documents/mf/momepy/momepy/shape.py:886: RuntimeWarning: invalid value encountered in arccos
  angle = np.degrees(np.arccos(cosine_angle))
  1%|▏         | 3587/252386 [00:03<09:38, 430.19it/s] /home/arch_ubuntu/miniconda/envs/momepy_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/arch_ubuntu/miniconda/envs/momepy_env/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 252386/252386 [03:50<00:00, 1096.98it/s]


"\ncencon = mm.CentroidCorners(blg)\n+blg['ssbCCM'] = cencon.mean\n+blg['ssbCCD'] = cencon.std\n"

In [5]:
blg['stbOri'] = mm.Orientation(blg).series
 
tess['stcOri'] = mm.Orientation(tess).series
'''
+blg['stbCeA'] = mm.CellAlignment(blg, tess, 'stbOri', 'stcOri', 'uID', 'uID').series
'''

100%|██████████| 252385/252385 [14:17<00:00, 294.42it/s]


"\n+blg['stbCeA'] = mm.CellAlignment(blg, tess, 'stbOri', 'stcOri', 'uID', 'uID').series\n"

In [6]:
tess['sdcLAL'] = mm.LongestAxisLength(tess).series
tess['sdcAre'] = mm.Area(tess).series
tess['sscCCo'] = mm.CircularCompactness(tess, 'sdcAre').series
tess['sscERI'] = mm.EquivalentRectangularIndex(tess, 'sdcAre').series

tess['sicCAR'] = mm.AreaRatio(tess, blg, 'sdcAre', 'sdbAre', 'uID').series
fa = mm.FloorArea(blg, 'sdbHei', 'sdbAre')
tess['sicFAR'] = mm.AreaRatio(tess, blg, 'sdcAre', fa.series, 'uID').series

100%|██████████| 252385/252385 [13:21<00:00, 315.03it/s]


In [7]:
blg["mtbSWR"] = mm.SharedWallsRatio(blg, "uID", "sdbPer").series
 
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tess, ids="uID")
 
blg["mtbAli"] = mm.Alignment(blg, queen_1, "uID", "stbOri").series
blg["mtbNDi"] = mm.NeighborDistance(blg, queen_1, "uID").series
tess["mtcWNe"] = mm.Neighbors(tess, queen_1, "uID", weighted=True).series
tess["mdcAre"] = mm.CoveredArea(tess, queen_1, "uID").series

100%|██████████| 252386/252386 [08:20<00:00, 504.55it/s]
/home/arch_ubuntu/miniconda/envs/momepy_env/lib/python3.7/site-packages/libpysal/weights/weights.py:165: UserWarning: The weights matrix is not fully connected: 
 There are 148 disconnected components.
  warnings.warn(message)
100%|██████████| 252385/252385 [02:20<00:00, 1793.29it/s]


In [8]:
blg_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blg)
 
blg["libNCo"] = mm.Courtyards(blg, "bID", blg_q1).series
blg["ldbPWL"] = mm.PerimeterWall(blg, blg_q1).series
 
blocks["ldkAre"] = mm.Area(blocks).series
blocks["ldkPer"] = mm.Perimeter(blocks).series
blocks["lskCCo"] = mm.CircularCompactness(blocks, "ldkAre").series
blocks["lskERI"] = mm.EquivalentRectangularIndex(blocks, "ldkAre", "ldkPer").series
blocks["lskCWA"] = mm.CompactnessWeightedAxis(blocks, "ldkAre", "ldkPer").series
blocks["ltkOri"] = mm.Orientation(blocks).series
 
blo_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blocks, ids="bID")
 
blocks["ltkWNB"] = mm.Neighbors(blocks, blo_q1, "bID", weighted=True).series
blocks["likWBB"] = mm.Count(blocks, blg, "bID", "bID", weighted=True).series
 
tess.to_file(path, layer="tessellation", driver="GPKG")
blg.to_file(path, layer="buildings", driver="GPKG")
blocks.to_file(path, layer="blocks", driver="GPKG")

/home/arch_ubuntu/miniconda/envs/momepy_env/lib/python3.7/site-packages/libpysal/weights/weights.py:165: UserWarning: The weights matrix is not fully connected: 
 There are 67436 disconnected components.
 There are 36260 islands with ids: 0, 1, 2, 3, 4, 5, 6, 7, 8, 68, 69, 70, 106, 173, 184, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 299, 306, 316, 590, 592, 606, 616, 623, 628, 679, 739, 746, 751, 756, 765, 795, 836, 842, 846, 847, 848, 849, 850, 851, 852, 855, 856, 857, 858, 859, 860, 861, 863, 988, 989, 990, 991, 992, 993, 994, 995, 997, 1010, 1011, 1031, 1422, 1574, 1576, 1604, 1640, 1651, 1662, 1705, 1768, 1855, 1856, 1868, 1873, 1878, 1883, 2088, 2192, 2193, 2194, 2195, 2196, 2197, 2200, 2201, 2202, 2203, 2204, 2205, 2206, 2207, 2208, 2211, 2212, 2213, 2217, 2218, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 223

In [9]:
queen1 = mm.sw_high(k=1, gdf=tess, ids='uID')
queen3 = mm.sw_high(k=3, weights=queen1)
blg_queen = blg_q1

blg['ltbIBD'] = mm.MeanInterbuildingDistance(blg, queen1, 'uID', queen3).series
blg['ltcBuA'] = mm.BuildingAdjacency(blg, queen3, 'uID', blg_queen).series

Generating adjacency matrix based on weights matrix...


  0%|          | 0/1635374 [00:00<?, ?it/s]

Computing interbuilding distances...


/home/arch_ubuntu/Documents/mf/momepy/momepy/distribution.py:661: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  (adj_list.neighbor == row.focal)
  0%|          | 0/252386 [00:00<?, ?it/s]

Computing mean interbuilding distances...


/home/arch_ubuntu/Documents/mf/momepy/momepy/distribution.py:684: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  adj_list.neighbor.isin(neighbours)
  0%|          | 0/252386 [00:00<?, ?it/s]

Calculating adjacency...


100%|██████████| 252386/252386 [00:39<00:00, 6327.20it/s]


In [10]:
blg['temp_fa'] = fa.series
tess = tess.merge(blg[['temp_fa', 'uID']], on='uID', how='left')
tess['licGDe'] = mm.Density(tess, 'temp_fa', queen3, 'uID', 'sdcAre').series
blg = blg.drop(columns='temp_fa')
tess = tess.drop(columns='temp_fa')
tess['ltcWRB'] = mm.BlocksCount(tess, 'bID', queen3, 'uID').series

100%|██████████| 252385/252385 [07:26<00:00, 565.19it/s]


In [11]:
tess.to_file(path, layer='tessellation', driver='GPKG')
blg.to_file(path, layer='buildings', driver='GPKG')
 
fo = libpysal.io.open('files/AMS/AMSqueen1.gal', 'w')
fo.write(queen1)
fo.close()
 
fo = libpysal.io.open('files/AMS/AMSqueen3.gal', 'w')
fo.write(queen3)
fo.close()
 
fo = libpysal.io.open('files/AMS/AMSblg_queen.gal', 'w')
fo.write(blg_queen)
fo.close()